<a href="https://colab.research.google.com/github/andrea-t94/airflow-net/blob/master/research/finetuning/notebooks/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Qwen2.5 on Airflow DAGs

This notebook demonstrates how to fine-tune the **Qwen2.5-1.5B-Instruct** model on a dataset of **Airflow DAGs** using [Unsloth](https://github.com/unslothai/unsloth) — a library that makes LLM fine-tuning 2x faster and uses 50% less memory.

### 🎯 Goal
Train a model that can generate valid Airflow DAGs from natural language instructions.

### 🛠️ Runtime Requirements
- **GPU**: Tesla T4 (Free Colab) or A100 (Colab Pro).
- **RAM**: Standard.

### 📋 Steps
1. **Setup**: Install dependencies.
2. **Config**: Define model and dataset parameters.
3. **Data**: Load and format the Airflow DAG dataset.
4. **Train**: Fine-tune with QLoRA (4-bit quantization).
5. **Save**: Push the fine-tuned model (GGUF & LoRA) to Hugging Face.

## 1. Setup & Installation
We install `unsloth` and other necessary libraries. If you are running this in Google Colab, it will automatically detect the environment and install the correct versions.

In [ ]:
%%capture
import os
import torch

# Check if running in Colab
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Install Unsloth and dependencies
if IN_COLAB:
    # Unsloth installation for Colab
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
else:
    # Standard installation
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Additional Requirements
!pip install datasets huggingface_hub

In [ ]:
# Verify GPU and PyTorch
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Detected: {gpu_name}")
    # Unsloth optimization info
    major_version, minor_version = torch.cuda.get_device_capability()
    if major_version >= 8:
        print("✅ GPU supports bfloat16 (Ampere or newer). Operations will be faster.")
    else:
        print("ℹ️ GPU is older than Ampere (e.g., T4). Using float16.")
else:
    print("❌ No GPU detected! Please change runtime type to GPU in 'Runtime > Change runtime type'.")

## 2. Configuration & Authentication
Log in to Hugging Face to access datasets and push your model.

In [ ]:
from huggingface_hub import login

# Try to get token from Colab secrets, otherwise prompt
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token, add_to_git_credential=True)
except (ImportError, KeyError, AttributeError):
    print("Please provide your Hugging Face Token (Permissions: Write)")
    login(add_to_git_credential=True)

In [ ]:
# Project Configuration
BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
DATASET_NAME = "andrea-t94/airflow-dag-dataset"

# Output Model Name (Change username if needed)
NEW_MODEL_NAME = "andrea-t94/qwen2.5-1.5b-airflow-instruct"

# Training Parameters
MAX_SEQ_LENGTH = 4096 # Fits most DAG files
LOAD_IN_4BIT = True   # Enable 4-bit quantization (QLoRA) to save memory

## 3. Load Model with Unsloth
We load the model in 4-bit precision to fit within standard Colab GPU memory (15GB).

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None, # Auto-detect float16 or bfloat16 based on GPU
    load_in_4bit = LOAD_IN_4BIT,
)

# Add LoRA (Low-Rank Adaptation) adapters
# This allows us to train only a small fraction of parameters (~1%)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,            # LoRA Rank (8, 16, 32, 64 are common)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Enable for long context
    random_state = 3407,
)

## 4. Load & Format Dataset
We specificy a formatting function to apply the ChatML template (which Qwen uses) to our dataset.

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset(DATASET_NAME)

# Inspect dataset sizes
print(f"Train size: {len(dataset['train'])}")
if 'eval' in dataset: print(f"Eval size:  {len(dataset['eval'])}")

# Format function for ChatML
# The dataset should have a 'messages' column matching standard chat format
def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        # Apply chat template but do NOT tokenize yet
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

## 5. Training
Configure the `SFTTrainer`. We use `gradient_accumulation_steps` to simulate a larger batch size.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset.get("eval"),
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Set to True to speed up training if sequences are short
    args = TrainingArguments(
        per_device_train_batch_size = 2,  # Increase if GPU memory allows
        gradient_accumulation_steps = 4,   # effective_batch = 2 * 4 = 8
        warmup_steps = 5,
        max_steps = 60,                   # Set to -1 for full epochs
        # num_train_epochs = 1,           # Uncomment for full training
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",             # Use 8-bit optimizer to save memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",               # Disable WandB for simplicity
    ),
)

In [ ]:
# Start Training
trainer_stats = trainer.train()

## 6. Save & Push to Hub
Unsloth allows saving both LoRA adapters (small) and the full merged model (large). We'll also push a GGUF version for local inference (e.g., with Ollama).

In [ ]:
# 1. Save LoRA Adapters only (Small file size, fast)
model.save_pretrained("lora_adapters")
model.push_to_hub(f"{NEW_MODEL_NAME}-lora", token=True)

# 2. Save Merged Model (Full model, slower but easier to use)
print("Saving merged model... this might take a while.")
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
model.push_to_hub_merged(NEW_MODEL_NAME, tokenizer, save_method = "merged_16bit", token=True)

In [ ]:
# 3. Convert to GGUF (for Ollama/Llama.cpp)
# Options: q4_k_m, q8_0, f16
print("Converting to GGUF...")
model.push_to_hub_gguf(
    NEW_MODEL_NAME,
    tokenizer,
    quantization_method = ["q4_k_m"],
    token = True
)